## 0) Get Data: DO NOT RUN TWICE !

### 0.0  Imports (only one )

In [0]:
import numpy as np
import os 

### 0.1 Mount Drive

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 1) Checkout Data

In [0]:
#1.0: mount the data part of the drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 1.0 *directories*

In [0]:
pretrain_dir='./gdrive/My Drive/Lip-Reading Project/Data/pretrain'
trainval_dir='./gdrive/My Drive/Lip-Reading Project/Data/trainval'
test_dir='./gdrive/My Drive/Lip-Reading Project/Data/test'

### 1.1 Number of Videos

In [0]:
import os
n_pretrain_vids=len(os.listdir(pretrain_dir))
n_trainval_vids=len(os.listdir(trainval_dir))
n_test_vids=len(os.listdir(test_dir))
print(f'The number of pretrain videos is {n_pretrain_vids}.\nThe number of trainval vidoes is {n_trainval_vids}.\nThe number of test videos is {n_test_vids}')

The number of pretrain videos is 5090.
The number of trainval vidoes is 4004.
The number of test videos is 451


### 1.2 Average number of sentences:

In [0]:
pretrain_vids=sorted(os.listdir(pretrain_dir))
trainval_vids=sorted(os.listdir(trainval_dir))
test_vids=sorted(os.listdir(test_dir))

In [0]:
#HOW did we generate those saved variables ?
#pretrain_sents=[len(os.listdir(pretrain_dir+'//'+x)) for x in pretrain_vids]
#trainval_sents=[len(os.listdir(trainval_dir+'//'+x)) for x in trainval_vids]
#test_sents=[len(os.listdir(test_dir+'//'+x)) for x in test_vids]
#np.save('./gdrive/My Drive/Lip-Reading Project/Data/pretrain_list',pretrain_sents)
#np.save('./gdrive/My Drive/Lip-Reading Project/Data/trainval_list',trainval_sents)
#np.save('./gdrive/My Drive/Lip-Reading Project/Data/test_list',test_sents)
##############################################################################
pretrain_sents=np.load('./gdrive/My Drive/Lip-Reading Project/Data/pretrain_list.npy')
trainval_sents=np.load('./gdrive/My Drive/Lip-Reading Project/Data/trainval_list.npy')
test_sents=np.load('./gdrive/My Drive/Lip-Reading Project/Data/test_list.npy')


In [0]:
print(f'Average number of sentences in pretrain is {np.mean(pretrain_sents)}, while the sum of sentences is: {np.sum(pretrain_sents)}')
print(f'Average number of sentences in trainval is {np.mean(trainval_sents)}, while the sum of sentences is: {np.sum(trainval_sents)}')
print(f'Average number of sentences in test is {np.mean(test_sents)}, while the sum of sentences is: {np.sum(test_sents)}')


Average number of sentences in pretrain is 23.28408644400786, while the sum of sentences is: 118516
Average number of sentences in trainval is 7.987512487512488, while the sum of sentences is: 31982
Average number of sentences in test is 3.2195121951219514, while the sum of sentences is: 1452


## 3) Preprocess Data:

### 3.0 Download youtube-dl and ffmpeg

In [0]:
#youtube-dl
!sudo apt-get install curl -y
!curl -L https://yt-dl.org/latest/youtube-dl -o /usr/bin/youtube-dl
!sudo chmod 755 /usr/bin/youtube-dl  

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   599    0   599    0     0    282      0 --:--:--  0:00:02 --:--:--   282
100 1721k  100 1721k    0     0   708k      0  0:00:02  0:00:02 --:--:-- 6258k


In [0]:
#ffmpeg
!sudo apt install ffmpeg
!pip install ffmpy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [0]:
from ffmpy import FFmpeg as fmpg

### 3.1 Download Youtube Videos

In [0]:
import subprocess as sb

def download_link_to_video(ref,vid_name):
  link="https://www.youtube.com/watch?v="+ref
  return sb.call(["youtube-dl",link,"--o",vid_name],shell=False) #downloads the video if it exists.
  #returns zero if successful, one if not. (oddly enough, zero is for success here)

#### 3.1.1 How much time will downloading the videos take ?

Since 10 videos took 12.0 minutes, then the 5090 videos
will take about 101.8 hours

In [0]:
import time
!mkdir vids

In [0]:
start=time.time()
refs_with_problem=[]
for yt_ref in pretrain_vids[0:10]:
  if(download_link_to_video(ref=yt_ref,vid_name="./vids/"+yt_ref)): #note that the if condition downloads the video !
    refs_with_problem.append(yt_ref)
    
end=time.time()
print(end-start)

703.2564492225647


In [0]:
minutes=np.round((end-start)/60)
print(f'Since 10 videos took {minutes} minutes, then the {len(pretrain_vids)} pretrain videos')
print(f'will take about {(len(pretrain_vids)/10)*minutes/60} hours')

Since 10 videos took 12.0 minutes, then the 5090 videos
will take about 101.8 hours


In [0]:
#Copy them to drive:
from shutil import copytree
copytree('./vids', './gdrive/My Drive/Lip-Reading Project/PresData/vids')

'./gdrive/My Drive/Lip-Reading Project/PresData/vids'

### 3.2 Edit the videos:

8 Videos took 26.78 seconds to be convertedto audio. It will take 5090 videos
37.86 hours to finish.

#### 3.2.2 How much time will converting 8 videos to audio take ?

In [0]:
start_aud=time.time()
for vid_name in vid_list:
  if(len(vid_name[:vid_name.rfind('.')])>0): #is a video and not an os arifcat
    fobj=fmpg(inputs={'./vids/'+vid_name:None},
                outputs={'./auds/'+vid_name[:vid_name.rfind('.')]+'.wav':None})
    fobj.run()
    #print(fobj.cmd)
end_aud=time.time()

In [0]:
print(f'8 Videos took {end_aud-start_aud} to be converted to audio. It will take {n_pretrain_vids} videos')
print(f'{n_pretrain_vids*(end_aud-start_aud)/(60*60)} hours to finish.')

8 Videos took 26.77778172492981 to be converted to audio. It will take 5090 videos
37.8608080499702 hours to finish.


In [0]:
#Copy to drive
from shutil import copytree
copytree('./auds', './gdrive/My Drive/Lip-Reading Project/PresData/auds')

'./gdrive/My Drive/Lip-Reading Project/PresData/auds'

### 3.3 Extract Sentences from Text
We have now 215 sentences.

In [0]:
###########################################33
def time_format(seconds):
    #uses HOURS:MM:SS.MILLISECONDS format
    m=seconds//60;
    hr=m//60;
    m=m-60*hr;
    s=seconds-m*60-hr*3600;
    m=round(m);
    hr=round(hr)
    s=round(s,3)

    time=str(hr)+':'+str(m)+':'+str(s)

    return time

In [0]:
pretrain_dir

'./gdrive/My Drive/Lip-Reading Project/Data/pretrain'

In [0]:
#GET LIST OF TEXT FILES FOR ALL THE SENTENCES IN THE VIDEOS (FULL VIDEOS, not sentences yet)

text_list=[]
for name in os.listdir('./gdrive/My Drive/Lip-Reading Project/PresData/vids'):
  if(len(name[:name.find('.')])>0): #is a video and not an os arifcat
    folder_name=name[:name.find('.')]
    for element in(sorted(os.listdir(pretrain_dir+'/'+folder_name))):
      text_list.append(pretrain_dir+'/'+folder_name+'/'+element)

In [0]:
print(text_list)
print(f'We have now {len(text_list)} sentences.')

['./gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00001.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00002.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00003.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00004.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00005.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00006.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00007.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00008.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00009.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00011.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00012.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00013.txt', './gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00014.txt'

In [0]:
file_path=text_list[0]
print(file_path)

./gdrive/My Drive/Lip-Reading Project/Data/pretrain/04SEzifEsGg/00001.txt


In [0]:
#### CREATE VIDEO: EXTENSION DICTIONARY ########
video_extension_dict={}
videos_folder= './gdrive/My Drive/Lip-Reading Project/PresData/vids'
for vid_name in os.listdir(videos_folder):
  if(vid_name.find('.')==0):
    continue
  video_extension_dict[vid_name[:vid_name.rfind('.')]]=vid_name[vid_name.rfind('.'):]
  
  
print(video_extension_dict)


{'04SEzifEsGg': '.mp4', '08ZWROqoTZo': '.mkv', '03h0dNZoxr8': '.mkv', '05jJodDVJRQ': '.mkv', '06M8qY7Q74Y': '.mkv', '00j9bKdiOjk': '.mkv', '04VbxnzUwes': '.webm', '0Amg53UuRqE': '.mkv'}


'.mp4'

In [0]:
#LOOP OVER TEXT FILES (SENTENCES) AND EXTRACT VIDEO
with open(file_path,'r') as f:
    file_contents=f.readlines()
    #########################################################
    #=======================================================#
    sub_file_path=file_path[file_path.rfind('/',0,-15):][1:-4]
    sent_file_name=sub_file_path[:sub_file_path.rfind('/')]+'_'+sub_file_path[sub_file_path.rfind('/')+1:]
    vid_name=sub_file_path[:sub_file_path.rfind('/')]+video_extension_dict[sub_file_path[:sub_file_path.rfind('/')]]
    print(f'sub_file_path: {sub_file_path},\nsent_file_name: {sent_file_name}\nvid_name:{vid_name}')
    #=======================================================#
    #########################################################
    print('\n'.join(file_contents))
    sentence=file_contents[0][7:-2]
    ref=file_contents[2][7:-1]
    start_seconds=int(file_contents[5][:6])/25
    print(sent_file_name,' : \n',sentence)
    
    for index,line in enumerate(file_contents[5:]):
      if (line[0]=='W'):
        #print(line)
        end_index=index+5
        break
    end_seconds=int(file_contents[end_index-2][:6])/25
    
    start_time=str(time_format(start_seconds))
    end_time=str(time_format(end_seconds))
    print(f'{start_time} to {end_time}')
    #######################################################################
    ####################EXTRACT SENTENCE with FFMPEG#######################
    #fobj=fmpg(inputs={'./gdrive/My Drive/Lip-Reading Project/PresData/vids/'+vid_name:None},
    #        outputs={'./sent_vids/'+sent_file_name+'.mkv':['-ss',start_time,'-to',end_time,'-vf','fps=']})
    #print(fobj.cmd)
    #fobj.run()
    #-vf fps=1 thumb%04d.jpg -hide_banner
 

sub_file_path: 04SEzifEsGg/00001,
sent_file_name: 04SEzifEsGg_00001
vid_name:04SEzifEsGg.mp4
Text:  IS WHERE ALL OF THIS COMES FROM AND THEN BEING AN ACTOR I WAS DOING THESE DIFFERENT KINDS OF THINGS AND I FELT THE CONTENT OF THE WORK THAT I WAS INVOLVED IN REALLY WASN'T CUTTING IT THAT THERE SURELY HAD TO BE 

Conf:  1

Ref:   04SEzifEsGg



FRAME 	X 	Y 	W 	H 

001159 	0.391 	0.152 	0.173 	0.441 

001160 	0.389 	0.153 	0.172 	0.451 

001161 	0.385 	0.156 	0.175 	0.456 

001162 	0.379 	0.152 	0.179 	0.457 

001163 	0.375 	0.143 	0.181 	0.450 

001164 	0.372 	0.128 	0.185 	0.452 

001165 	0.369 	0.109 	0.184 	0.443 

001166 	0.363 	0.100 	0.188 	0.439 

001167 	0.364 	0.096 	0.183 	0.452 

001168 	0.365 	0.099 	0.183 	0.458 

001169 	0.360 	0.095 	0.184 	0.468 

001170 	0.355 	0.092 	0.179 	0.458 

001171 	0.350 	0.081 	0.183 	0.456 

001172 	0.348 	0.094 	0.174 	0.417 

001173 	0.346 	0.066 	0.186 	0.463 

001174 	0.345 	0.085 	0.166 	0.432 

001175 	0.340 	0.086 	0.164 	0.432 

001176

In [0]:
#LOOP OVER TEXT FILES (SENTENCES) AND EXTRACT VIDEO
for file_path in text_list:
  with open(file_path,'r') as f:
    file_contents=f.readlines()
    #########################################################
    #=======================================================#
    sub_file_path=file_path[file_path.rfind('/',0,-15):][1:-4]
    sent_file_name=sub_file_path[:sub_file_path.rfind('/')]+'_'+sub_file_path[sub_file_path.rfind('/')+1:]
    aud_name=sub_file_path[:sub_file_path.rfind('/')]+'.wav'
    #=======================================================#
    #########################################################
    #print(file_contents)
    sentence=file_contents[0][7:-2]
    ref=file_contents[2][7:-1]
    start_seconds=int(file_contents[5][:6])/25
    print(sent_file_name,' : \n',sentence)
    #print(ref)
    #print(start_seconds)
    for index,line in enumerate(file_contents[5:]):
      if (line[0]=='W'):
        #print(line)
        end_index=index+5
        break
    end_seconds=int(file_contents[end_index-2][:6])/25
    #print(f'{time_format(start_seconds)}-{time_format(end_seconds)}')

    start_time=str(time_format(start_seconds))
    end_time=str(time_format(end_seconds))
    #print(f'{start_time} to {end_time}')
    #######################################################################
    ####################EXTRACT SENTENCE with FFMPEG#######################
    fobj=fmpg(inputs={'./gdrive/My Drive/Lip-Reading Project/PresData/vids/'+aud_name:None},
            outputs={'./sent_auds/'+sent_file_name+'.wav':['-ss',start_time,'-to',end_time,'-c','copy']})
    fobj.run()

03h0dNZoxr8_00017  : 
 TO HAPPEN TO YOU AND ONLY 1 IN 10 PEOPLE WHO ARE OVER 80 WILL DIE OF CANCER THE BIG
03h0dNZoxr8_00018  : 
 BIGGEST GROWTH INDUSTRY OF ALL AND AT LEAST 6 OUT OF 10 OF THE PEOPLE IN THIS ROOM WILL DIE IN THIS FORM
03h0dNZoxr8_00019  : 
 THE DWINDLING OF CAPACITY WITH
03h0dNZoxr8_00020  : 
 WE ARE ALL MOST OF US LIVING TO REACH THIS POINT HISTORICALLY WE DIDN'T DO THAT THIS IS WHAT HAPPENS TO YOU WHEN YOU LIVE TO BE AT
03h0dNZoxr8_00021  : 
 UNFORTUNATELY INCREASING LONGEVITY DOES MEAN MORE OLD AGE NOT MORE YOUTH I'M SORRY TO SAY THAT
03h0dNZoxr8_00022  : 
 TO ASK PEOPLE BECAUSE GIVING PEOPLE THE CONTROL OF WHO THAT IS PRODUCES AN AMAZING OUTCOME THE SECOND THING YOU CAN SAY IS HAVE YOU SPOKEN TO THAT
03h0dNZoxr8_00023  : 
 I ACTUALLY THINK THAT IN COUNTRIES IN PLACES LIKE OREGON WHERE YOU CAN HAVE PHYSICIAN ASSISTED SUICIDE YOU TAKE A POISONOUS DOSE OF
03h0dNZoxr8_00024  : 
 ONLY HALF A PERCENT OF PEOPLE EVER DO THAT I'M MORE INTERESTED IN WHAT HAPPENS TO THE 99 5 

In [0]:
#Copy to drive
from shutil import copytree
copytree('./sent_auds', './gdrive/My Drive/Lip-Reading Project/PresData/sent_auds')

'./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds'

### 3.4 Extract words from Sentences:

Here, I try that, given the sent_auds folder and the text files only, to extract the words.

In [0]:
!mkdir sent_words

In [0]:
sent_files=os.listdir('./sent_auds')
sent_aud_file=sent_files[0]

for sent_aud_file in sent_files:

  sent_aud_file_path='./sent_auds/'+sent_aud_file

  #New Folder name:
  sent_folder_name=sent_aud_file[:sent_aud_file.rfind('.')]
  sent_folder_path="./sent_words/"+sent_folder_name
  #Old Variables referred to:
  ref=sent_aud_file[:sent_aud_file.find('_')]
  txt_f_name=sent_aud_file[sent_aud_file.find('_')+1:sent_aud_file.rfind('.')]+'.txt'
  print(ref)
  print(text_file)
  ######################################################################
  sb.call(["mkdir",sent_folder_path],shell=False)
  ######################################################################
  #Find end of frames:
  with open(pretrain_dir+'/'+ref+'/'+txt_f_name,'r') as f:
    file_contents=f.readlines()
    for index,line in enumerate(file_contents[5:]):
        if (line[0]=='W'):
          #print(line)
          end_index=index+5
          break
  cntr=0
  for line in file_contents[end_index+1:]: #WRITE WORDS TO WAV FILES IN THIS LOOP
    word_info_list=line.split()
    word,start_sec,end_sec=(word_info_list[0],word_info_list[1],word_info_list[2])
    start_time=time_format(float(start_sec))
    end_time=time_format(float(end_sec))
    start_sec_reversed=start_sec[::-1]#just used to name the word audiofile to avoid replicas
    word_aud_name=word+"_"+str(cntr)+".wav"
    print(f'{word_aud_name}')
    #############################EXTRACT AUDIO##########################
    fobj=fmpg(inputs={sent_aud_file_path:None},
          outputs={sent_folder_path+'/'+word_aud_name:['-ss',start_time,'-to',end_time,'-c','copy']})
    fobj.run()
    cntr=cntr+1
    #print(fobj.cmd)

#### Sorter Example:

In [0]:
def sent_word_sorter(audio_file_name):
  without_ext=audio_file_name[:audio_file_name.rfind('.')]
  return int(without_ext[without_ext.rfind('_')+1:])

In [0]:
word_auds=os.listdir('./sent_words/00j9bKdiOjk_00001')
word_auds=sorted(word_auds,key=sent_word_sorter)

#### Maintain A copy in drive:

In [0]:
#Copy to drive
from shutil import copytree
copytree('./sent_words', './gdrive/My Drive/Lip-Reading Project/PresData/sent_words')

'./gdrive/My Drive/Lip-Reading Project/PresData/sent_words'

### 3.5 Create An array of paramters for evey sentence: 

### 3.5.1 Get Features:

#### 3.5.1.1 Explore:

In [0]:
!pip install python_speech_features

  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import numpy as np
import scipy.io.wavfile as wav
import python_speech_features

In [0]:
fs, signal = wav.read('./gdrive/My Drive/Lip-Reading Project/PresData/sent_words/00j9bKdiOjk_00001/PROTECTION_25.wav')

In [0]:
print("fs: ",fs,", length of signal: ",len(signal))

fs:  48000 , length of signal:  21504


In [0]:
signal=signal[::int(np.round(48000/100))]
print(len(signal))

45


In [0]:
#reference(edited): https://github.com/WindQAQ/listen-attend-and-spell/blob/master/vctk/preprocess.py

from python_speech_features import mfcc

def extract_mfcc(filename):
    if not filename.endswith('.wav'):
        return None, None

    fs, signal = wav.read(filename)
    
    # downsample to 100 Hz
    signal=signal[::int(np.round(fs/100))]
    print("length of signal now:",len(signal))
    fs=100
#     signal = signal[::3]
#     fs = fs/3
    
    if(signal.shape[1]==2):
      signal=(np.sum(signal,axis=1))/2
      #print(signal.shape)
    
    mfcc_feat= mfcc(signal, samplerate=fs, winlen=0.025, winstep=0.025, numcep=13)
    mfcc_feat_norm=mfcc_feat
    return mfcc_feat_norm

In [0]:
#parameters=extract_mfcc('./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/00j9bKdiOjk_00001.wav')
filename='./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/00j9bKdiOjk_00001.wav'
fs, signal = wav.read(filename)
print("fs:",fs,", Shape: ",signal.shape,", length: ",len(signal))
if(signal.shape[1]==2):
      signal=(np.sum(signal,axis=1))/2
print("fs:",fs,", Shape: ",signal.shape,", length: ",len(signal))
signal=signal[::int(np.round(fs/100))]
fs=100
print("fs:",fs,", Shape: ",signal.shape,", length: ",len(signal))
    
#print(parameters.shape)

fs: 48000 , Shape:  (476160, 2) , length:  476160
fs: 48000 , Shape:  (476160,) , length:  476160
fs: 100 , Shape:  (992,) , length:  992


In [0]:
from python_speech_features import mfcc
mfcc_feat = mfcc(signal, samplerate=100, winlen=0.025, winstep=0.025, numcep=13)

In [0]:
mfcc_feat.shape

(331, 13)

In [0]:
import os
frames=[]
path='./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds'

for sent_aud in os.listdir('./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds'):
  if(sent_aud.find('.')>0):
    feats=extract_mfcc('./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds'+'/'+sent_aud)
    frames.append(feats.shape[0])
    
  

length of signal now: 1197
length of signal now: 1302
length of signal now: 1669
length of signal now: 2089
length of signal now: 1150
length of signal now: 1908
length of signal now: 2025
length of signal now: 1312
length of signal now: 2473
length of signal now: 2240
length of signal now: 1925
length of signal now: 1980
length of signal now: 1376
length of signal now: 1592
length of signal now: 2189
length of signal now: 2409
length of signal now: 2176
length of signal now: 1462
length of signal now: 3247
length of signal now: 1417
length of signal now: 3092
length of signal now: 1167
length of signal now: 2511
length of signal now: 1880
length of signal now: 713
length of signal now: 1564
length of signal now: 732
length of signal now: 658
length of signal now: 1024
length of signal now: 2066
length of signal now: 3770
length of signal now: 992
length of signal now: 389
length of signal now: 431
length of signal now: 378
length of signal now: 512
length of signal now: 847
length of 

In [0]:
print(np.max(frames)) ##MAXIMUM NUMBER OF FRAMES=3767

1257


#### 3.5.1.2 Implement:

In [0]:
! mkdir './sent_auds/train_data'

In [0]:
#remember:if(sent_aud.find('.')>0):
max_len_frames=3767
n_cebestral=13
sent_auds= os.listdir('./sent_auds')
for sent_aud in sent_auds:
  if(sent_aud.find('.')>0):
    sent_aud_path='./sent_auds/'+sent_aud
    sent_aud_train_path='./sent_auds/train_data/'+sent_aud[:sent_aud.rfind('.')]
    paramters=extract_mfcc(sent_aud_path)

    padding=np.repeat(np.zeros((1,n_cebestral)), max_len_frames-paramters.shape[0], axis=0)
    padded=np.concatenate((paramters,padding),axis = 0)
    np.save(sent_aud_train_path,padded)

In [0]:
#MAINTAIN A COPY:
#Copy to drive
from shutil import copytree
copytree('./sent_auds/train_data', './gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/train_data')

'./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/train_data'

#### 3.5.1.3 Make Labels:

In [0]:
! mkdir './sent_auds/char_label_data'

In [0]:
train_data=os.listdir(path+'/train_data')
for file_name in train_data:
  witout_ext=file_name[:file_name.rfind('.')]
  folder_name=file_name[:file_name.find('_')]
  text_file_name=witout_ext[witout_ext.find('_')+1:]+'.txt'
  file_path=pretrain_dir+'/'+folder_name+'/'+text_file_name
  with open(file_path,'r') as f:
    file_contents=f.readlines()
    sentence=file_contents[0][7:-2]+'$'
    np.save(path+'/char_label_data/'+witout_ext,sentence)

In [0]:
#MAINTAIN A COPY:
#Copy to drive
from shutil import copytree
copytree('./sent_auds/char_label_data', './gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/char_label_data')

'./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/char_label_data'

#### 3.5.1.4 Max Length of labels:

In [0]:
len_sents=[]
train_data=os.listdir('./sent_auds/train_data')
for file_name in train_data:
  witout_ext=file_name[:file_name.rfind('.')]
  folder_name=file_name[:file_name.find('_')]
  text_file_name=witout_ext[witout_ext.find('_')+1:]+'.txt'
  file_path=pretrain_dir+'/'+folder_name+'/'+text_file_name
  with open(file_path,'r') as f:
    file_contents=f.readlines()
    sentence=file_contents[0][7:-2]+'$'
    len_sents.append(len(sentence))

In [0]:
print(np.max(len_sents))

560


In [0]:
MAX_SENT = 560
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789' $"
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

labels=os.listdir('./sent_auds/char_label_data')
for file_name in labels:
  witout_ext=file_name[:file_name.rfind('.')]
  folder_name=file_name[:file_name.find('_')]
  text_file_name=witout_ext[witout_ext.find('_')+1:]+'.txt'
  file_path=pretrain_dir+'/'+folder_name+'/'+text_file_name
  data=str(np.load('./sent_auds/char_label_data/'+file_name))
  print(data)
  integer_encoded = [char_to_int[char] for char in data]
  # one hot encode
  onehot_encoded = list()
  for i in range(0,MAX_SENT):
    letter = [0 for _ in range(len(alphabet))]
    if (i < len(integer_encoded)):
      letter[integer_encoded[i]] = 1
      onehot_encoded.append(letter)
    else:
      letter[len(alphabet)-1] = 1
      onehot_encoded.append(letter)
  print(onehot_encoded)
  np.save('./sent_auds/label_data/'+witout_ext,onehot_encoded) 

AND A DIFFERENT WAY OF APPROACHING PROBLEMS WHEN YOU'RE TRYING TO DO THINGS$
[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: ignored

In [0]:
#MAINTAIN A COPY:
#Copy to drive
from shutil import copytree
copytree('./sent_auds/label_data', './gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/label_data')

'./gdrive/My Drive/Lip-Reading Project/PresData/sent_auds/label_data'

In [0]:
print(labels)

['06M8qY7Q74Y_00002.npy', '05jJodDVJRQ_00013.npy', '05jJodDVJRQ_00017.npy', '03h0dNZoxr8_00005.npy', '06M8qY7Q74Y_00018.npy', '04VbxnzUwes_00012.npy', '06M8qY7Q74Y_00003.npy', '03h0dNZoxr8_00002.npy', '08ZWROqoTZo_00008.npy', '04SEzifEsGg_00009.npy', '0Amg53UuRqE_00003.npy', '04SEzifEsGg_00058.npy', '05jJodDVJRQ_00005.npy', '04SEzifEsGg_00030.npy', '08ZWROqoTZo_00032.npy', '04SEzifEsGg_00012.npy', '05jJodDVJRQ_00027.npy', '04VbxnzUwes_00003.npy', '04SEzifEsGg_00005.npy', '06M8qY7Q74Y_00007.npy', '04SEzifEsGg_00057.npy', '05jJodDVJRQ_00029.npy', '08ZWROqoTZo_00011.npy', '08ZWROqoTZo_00034.npy', '04SEzifEsGg_00015.npy', '03h0dNZoxr8_00011.npy', '04VbxnzUwes_00013.npy', '08ZWROqoTZo_00020.npy', '04VbxnzUwes_00024.npy', '03h0dNZoxr8_00014.npy', '03h0dNZoxr8_00012.npy', '04VbxnzUwes_00018.npy', '04SEzifEsGg_00053.npy', '08ZWROqoTZo_00031.npy', '04SEzifEsGg_00038.npy', '08ZWROqoTZo_00010.npy', '04SEzifEsGg_00029.npy', '04SEzifEsGg_00063.npy', '04SEzifEsGg_00026.npy', '05jJodDVJRQ_00021.npy',

In [0]:
labels[0]

'06M8qY7Q74Y_00002.npy'